# Cropping outsider

In [1]:
import numpy as np
import cv2

In [2]:
body_cascade = cv2.CascadeClassifier('haarcascade_fullbody.xml')

SystemError: <class 'cv2.CascadeClassifier'> returned a result with an error set

In [3]:
input_img=cv2.imread('Vriksasana.jpeg', cv2.IMREAD_GRAYSCALE) # get the array of the original pic
print(input_img)
faces = body_cascade.detectMultiScale(input_img, 
                                      scaleFactor=1.1,
                                      minNeighbors=5)
input_img.shape




[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]


NameError: name 'body_cascade' is not defined

In [17]:
x,y,w,h = faces[0]
img_data= input_img[y:y+h,x:x+w]
img_data=cv2.resize(img_data,(48,48))
    
img_data = np.stack(img_data)
img_data = img_data / 255.0

NameError: name 'faces' is not defined